In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install biosignalsnotebooks
from h5py import File
import numpy as np
from numpy import array
import biosignalsnotebooks as bsnb
import pandas as pd
from bokeh.plotting import figure, show

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.3 MB/s eta 0:00:00
  Created wheel for libmagic: filename=libmagic-1.0-py3-none-any.whl size=4269 sha256=481214ded4ef0702d76e8dd452af9111e3d00df18f12ec94a68b897e93ca33ca
  Stored in directory: /root/.cache/pip/wheels/31/ce/d8/099fbd4dbfa5da5596a39eee8a17d2e1bcc29006c07d3563b7
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=92a830aa44792be9373e21ba702e327bbad3d385aebf9e56f6ed1404f6a24ed7
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582

In [4]:
File_Name = "pregunta2_simple.h5"
h5_object = File(File_Name)
available_keys = list(h5_object.keys())
print(available_keys)
mac = available_keys[0]
group_lv1 = h5_object.get(mac)
attrs_lv1 = group_lv1.attrs.items()
header_h5 = dict(attrs_lv1)
print(header_h5)

data,header = bsnb.load("pregunta2_simple.h5",get_header=True)
signal = data["CH4"]

time = bsnb.generate_time(signal,header["sampling rate"])
sr = header["sampling rate"]
sampling_rate = sr
resolution = header["resolution"][0]

#Ventana de tiempo
t_start = 0
sample_start = t_start*sampling_rate

t_end = 30
sample_end = t_end*sampling_rate

#Proceed to the unit conversion and generate a time-axis
signal_uv = bsnb.raw_to_phy("EEG", "biosignalsplux", signal, resolution, "uV")

['98:D3:51:FD:71:01']
{'channels': array([4], dtype=int32), 'comments': '', 'date': '2023-4-19', 'device': 'bitalino_rev', 'device connection': '98:D3:51:FD:71:01', 'device name': '98:D3:51:FD:71:01', 'digital IO': array([0, 0, 1, 1], dtype=int32), 'duration': '36s', 'firmware version': 1282, 'keywords': '', 'macaddress': '98:D3:51:FD:71:01', 'mode': 0, 'nsamples': 36750, 'resolution': array([ 4,  1,  1,  1,  1, 10], dtype=int32), 'sampling rate': 1000, 'sync interval': 2, 'time': '12:6:13.58'}


In [5]:
#Frecuencias de corte
low_cutoff_wide = 3 #lower cutoff frequency for bandpass filter (Hz)
high_cutoff_wide = 30 #Upper cutoff frequency for bandpass filter (Hz)

# Baseline shift of window
signal_shift_window = array(signal_uv[sample_start:sample_end]) - np.mean(array(signal_uv[sample_start:sample_end]))


In [6]:
#Frecuencias de corte
filtered_signal_3_30 = bsnb.bandpass(signal_shift_window, low_cutoff_wide, high_cutoff_wide, order = 2, fs = sr)
signal = data["CH4"]
time = bsnb.generate_time(signal,header["sampling rate"])

In [7]:
#Nueva Bokeh figure
bokeh_figure = figure(x_axis_label = "Time(s)", y_axis_label = "Raw Data")

In [8]:
bokeh_figure.line(time, signal, legend_label="Datos originales")

GlyphRenderer(id='1037', ...)

In [9]:
show(bokeh_figure)

In [10]:
signal_acoustic = data["CH4"]
signal_eeg = data["CH4"]

device = header["device"]

signal_uv = bsnb.raw_to_phy("EEG",device,signal_eeg,resolution,option = "uV") #Conversion a uV

#Estimulo:
signal_ac = signal_acoustic - np.mean(signal_acoustic)

#Generando los ejes de tiempo:
time_eeg = bsnb.generate_time(signal_uv)
time_a = bsnb.generate_time(signal_acoustic)

time_a_down = time_a[::10]
signal_ac_down = signal_ac[::10]

#Estimulo sonoro:
bsnb.plot([time_a_down],[signal_ac_down],y_axis_label="Value RAW", legend = "Acoustic Stimuli RAW", x_range = (0,310))

#Conversion de unidades
signal_uv = bsnb.raw_to_phy("EEG","biosignalsplux",signal_eeg, resolution, "uV")
time_uv = bsnb.generate_time(signal_uv,sr)
